In [ ]:
# Importing necessary libraries
import pydicom
import numpy as np
import matplotlib.pyplot as plt
import cv2  # OpenCV for resizing images

# Function to load and preprocess a DICOM image
def preprocess_dicom(file_path):
    # Load the DICOM file
    dicom = pydicom.dcmread(file_path)
    
    # Extract the pixel array from the DICOM file
    image = dicom.pixel_array
    
    # Normalize the pixel values to the range [0, 1]
    image = image.astype(np.float32)
    image -= np.min(image)
    image /= np.max(image)
    
    # Resize the image to a standard size (e.g., 256x256)
    image_resized = cv2.resize(image, (256, 256))
    
    return image_resized

# Function to visualize the image
def visualize_image(image, title='DICOM Image'):
    plt.figure(figsize=(6, 6))
    plt.imshow(image, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

# List of file paths (for demonstration, using the provided file paths)
file_paths = [
    '/mnt/data/1.2.276.0.7230010.3.1.4.8323329.300.1517875162.258081.dcm',
    '/mnt/data/1.2.276.0.7230010.3.1.4.8323329.301.1517875162.280319.dcm',
    '/mnt/data/1.2.276.0.7230010.3.1.4.8323329.302.1517875162.286330.dcm'
]

# Preprocess and visualize each image
for file_path in file_paths:
    preprocessed_image = preprocess_dicom(file_path)
    visualize_image(preprocessed_image, title=f'Preprocessed {file_path}')
